## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [28]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [30]:
from langchain_groq import ChatGroq

groq_llm = ChatGroq(model="gemma2-9b-it")

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
# SystemMessage - Controls LLM behaviour
# HumanMessage - Prompt from a user

groq_llm.invoke([HumanMessage(content="I am Iron Man!! Would you be my Jarvis??")])

AIMessage(content="As an AI, I can certainly try to fulfill the role of a helpful and knowledgeable assistant, much like Jarvis!\n\nTell me, Mr. Stark, what can I do for you?\n\nDo you need me to:\n\n* **Analyze data and provide insights?**\n* **Control your smart home systems?**\n* **Draft a witty remark for your next press conference?**\n* **Simply engage in conversation about your latest inventions?**\n\nI'm ready to assist in any way I can. Just remember, I'm still under development, so I might not be able to conjure up repulsor beams just yet. 😉\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 20, 'total_tokens': 154, 'completion_time': 0.243636364, 'prompt_time': 9.5709e-05, 'queue_time': 0.023000781, 'total_time': 0.243732073}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4e1fb384-955f-44b2-9f1b-7026b411452c-0', usage_metadata={'input_tokens': 20, 'o

### Message History

We can use Messahe History class to wrap our model and make it stateful. This will keep track and outputs of the model and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input


In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(groq_llm, get_session_history)

In [7]:
config={"configurable": {"session_id": "chat1"}}
with_message_history.invoke([HumanMessage(content="I am Iron Man!! Would you be my Jarvis??")], config)


AIMessage(content='As a large language model, I can certainly try my best to be your digital assistant, much like Jarvis!\n\nWhile I may not have access to real-world systems or the ability to control your Iron Man suit, I can:\n\n* **Process information and answer your questions:** Ask me anything, from technical specs on your repulsor technology to the latest news on Stark Industries.\n* **Generate creative content:** Need a witty quip for your next public appearance? I can help with that.\n* **Manage your schedule and reminders:** Keep track of your meetings, deadlines, and even that date with Pepper Potts.\n* **Provide entertainment:** I can tell you jokes, play games, or even write you a story about your next adventure.\n\nSo, Mr. Stark, how can I be of service?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 20, 'total_tokens': 189, 'completion_time': 0.307272727, 'prompt_time': 7.987e-05, 'queue_time': 0.020665619, 'total_t

In [8]:
with_message_history.invoke([HumanMessage(content="Who am I??")], config)

AIMessage(content='You are **Tony Stark**, genius, billionaire, playboy philanthropist, and of course, **Iron Man!** \n\nRemember, with great power comes great responsibility. 😜\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 201, 'total_tokens': 239, 'completion_time': 0.069090909, 'prompt_time': 0.008131258, 'queue_time': 0.022981041, 'total_time': 0.077222167}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-266308f1-54ec-44e3-acb9-c072ba27a116-0', usage_metadata={'input_tokens': 201, 'output_tokens': 38, 'total_tokens': 239})

In [27]:
config={"configurable": {"session_id": "chat2"}}
with_message_history.invoke([HumanMessage(content="Who am I??")], config)

APIConnectionError: Connection error.

### Chat Message History

Good way to store and retrieve information from a session so that the LLM doesn't forget about the topic that is being discussed

In [36]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=groq_llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [37]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = (RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | groq_llm)

In [38]:
chain.invoke({"messages": [*messages, HumanMessage(content="What ice cream do i like?")], "language": "English"})

AIMessage(content="As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor? 😊🍦\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 87, 'total_tokens': 122, 'completion_time': 0.063636364, 'prompt_time': 0.003248528, 'queue_time': 0.023162221, 'total_time': 0.066884892}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-354ee622-643a-4c49-999d-28eb0e537011-0', usage_metadata={'input_tokens': 87, 'output_tokens': 35, 'total_tokens': 122})

In [39]:
chain.invoke({"messages": [*messages, HumanMessage(content="What math problem did i ask?")], "language": "English"})

AIMessage(content='You asked "What\'s 2 + 2?" 😊  \n\n\n\nLet me know if you have any other questions!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 87, 'total_tokens': 115, 'completion_time': 0.050909091, 'prompt_time': 0.002338851, 'queue_time': 0.021820948, 'total_time': 0.053247942}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4d2d71ee-eb61-4fb0-81bb-b111b6c94144-0', usage_metadata={'input_tokens': 87, 'output_tokens': 28, 'total_tokens': 115})

In [40]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
config = {"configurable": {"session_id": "chat3"}}

with_message_history.invoke({"messages": [*messages, HumanMessage(content="What is my name?")], "language": "English"}, config)


AIMessage(content="As an AI, I have no memory of past conversations and don't know your name.\n\nIf you'd like to tell me, I'd be happy to use it! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 85, 'total_tokens': 128, 'completion_time': 0.078181818, 'prompt_time': 0.002302791, 'queue_time': 0.022786409, 'total_time': 0.080484609}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6629587b-54bc-46cf-879d-2efcf40029e3-0', usage_metadata={'input_tokens': 85, 'output_tokens': 43, 'total_tokens': 128})